In [19]:
#!/usr/bin/python
import torch
import os
import pandas as pd
import yaml

from deeplab3.dataloaders import make_data_loader
from deeplab3.config.defaults import get_cfg_defaults
from deeplab3.test import Tester

In [8]:
# traverse root directory, and list directories as dirs and files as files
model_configs = []
for root, dirs, files in os.walk("../models/"):
    path = root.split(os.sep)
    for file in files:
        if file == "parameters.txt.yaml" or file=="parameters.yaml":
            model_configs.append(os.path.join(path, file))
print(model_configs)

['../models/run/cityscapes/cityscapes_depthaware_coarse/2020_08_07-11_41_31/parameters.yaml', '../models/run/cityscapes/cityscapes_depthaware_coarse/2020_08_07-11_48_47/parameters.yaml', '../models/run/cityscapes/cityscapes_depthaware_fine/2020_08_04-17_18_18/parameters.yaml', '../models/run/cityscapes/cityscapes_depthaware_fine_coarse/2020_08_06-16_44_58/parameters.yaml', '../models/run/cityscapes/cityscapes_depthaware_fine_coarse/2020_08_09-10_04_31/parameters.yaml', '../models/run/cityscapes/cityscapes_rgb_xception_pt_fine_coarse/2020_08_03-15_41_22/parameters.yaml', '../models/run/cityscapes/cityscapes_rgbd_resnet_coco_fine/2020_08_05-18_17_41/parameters.yaml', '../models/run/cityscapes/cityscapes_rgbd_resnet_coco_fine/2020_08_06-23_14_45/parameters.yaml', '../models/run/cityscapes/cityscapes_rgbd_resnet_coco_fine/2020_08_07-14_44_47/parameters.yaml', '../models/run/cityscapes/cityscapes_rgbd_resnet_coco_fine/2020_08_07-17_11_09/parameters.yaml', '../models/run/cityscapes/cityscape

In [40]:
def flatten_cfg(cfg_dict, prefix=None):
    flattened = {}
    for key, value in cfg_dict.items():
        if prefix is None:
            prefixed_key = key
        else:
            prefixed_key = "{}.{}".format(prefix, key)
            
        if not isinstance(value, dict):
            flattened[prefixed_key] = value
        else:
            flattened.update(flatten_cfg(value, prefixed_key))
    return flattened

In [44]:
model_output = {}
for cfg_filepath in model_configs:
    cfg = get_cfg_defaults()
    with open(cfg_filepath, 'r') as f:
        model_cfg = yaml.load(f)
        model_cfg = flatten_cfg(model_cfg)
    for key, value in model_cfg.items():
        if not key.startswith("CHECKPOINT"):
            try:
                cfg.merge_from_list([key, value])
            except ValueError as e:
                print(e)
                
    cfg.merge_from_list(['CHECKPOINT.DIRECTORY', os.path.dirname(cfg_filepath)])
    if cfg.DATASET.ROOT.startswith("datasets"):
        cfg.merge_from_list(['DATASET.ROOT', "../" + cfg.DATASET.ROOT])

    torch.manual_seed(cfg.SYSTEM.SEED)
    train_loader, val_loader, test_loader, num_classes = make_data_loader(cfg)
    tester = Tester(cfg)
    output, mat = tester.run(val_loader, num_classes)
        
    model_output[cfg_filepath] = output
    break

/Users/Mauceri/anaconda/envs/detectron2/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


Type mismatch (<class 'float'> vs. <class 'int'>) with values (1.0 vs. 1) for config key: TRAIN.EVAL_INTERVAL
Using RGB-D input
Found 19997 train_extra images
Using RGB-D input
Found 500 val images
Using RGB-D input
Found 1525 test images
Training backbone from scratch


AssertionError: Torch not compiled with CUDA enabled

In [46]:
print(cfg)

CHECKPOINT:
  DIRECTORY: ../models/run/cityscapes/cityscapes_depthaware_coarse/2020_08_07-11_41_31
  EXCLUDE: []
  MODEL: checkpoint.pth.tar
  RESUME: False
DATASET:
  BASE_SIZE: 513
  CITYSCAPES:
    DEPTH_DIR: disparity
    GT_MODE: gtCoarse
    TRAIN_SET: train_extra
  COCO:
    CATEGORIES: coco
  CROP_SIZE: 513
  DARKEN: False
  MODE: RGBD
  NAME: cityscapes
  N_CLASSES: 19
  POWER_TRANSFORM: False
  PT_LAMBDA: -0.5
  ROOT: ../datasets/cityscapes/
  SYNTHETIC: False
  USE_STD: True
EVALUATOR:
  NAME: segmentation
MODEL:
  ASPP_DOUBLE: False
  BACKBONE: depthaware_resnet
  BACKBONE_ZOO: False
  DECODER_DOUBLE: False
  FREEZE_BN: False
  INPUT_CHANNELS: 4
  LOSS_TYPE: ce
  MOBILENET:
    WIDTH_MULT: 1.0
  NAME: deeplab
  OUT_STRIDE: 16
  PRETRAINED: 
  RESNET:
    DEEPLAB_OUTPUT: True
    OUT_FEATURES: ['res5', 'res2']
  SYNC_BN: False
  VGG16:
    BN: True
    DEPTH_CONV: True
SYSTEM:
  CUDA: True
  GPU_IDS: [1]
  NO_CUDA: False
  NUM_WORKERS: 4
  SEED: 1
TEST:
  BATCH_SIZE: 4
TRAIN

In [79]:
pd.options.display.max_colwidth = 35
pd.options.display.max_rows = 100

df = pd.DataFrame(model_results)
df = df.sort_values('value', ascending=False)

with open('all_models.csv', 'w') as fp:
    df.to_csv(fp)

In [80]:
models = ['xception', 'resnet']
datasets = ['cityscapes', 'coco', 'sunrgbd']
modes = ['RGB', "RGBD", 'RGB_HHAmid', 'RGB_HHAlat']

with open('top_models.csv', 'w') as fp:
    header = True
    for model in models:
        for data in datasets:
            for mode in modes:
                if len(df[((df['model'] == model) & (df['dataset'] == data) & (df['mode'] == mode))].index) > 1:
                    row = df[((df['model'] == model) & (df['dataset'] == data) & (df['mode'] == mode))]
                    indices.append(row.index[0])
                    print(row.head(1))
                    row.head(1).to_csv(fp, header=header)
                    header = False

       model     dataset mode                                path     value
57  xception  cityscapes  RGB  ../models/temp/models/cityscape...  0.763971
       model     dataset  mode                                path     value
67  xception  cityscapes  RGBD  ../models/temp/models/cityscape...  0.780869
       model     dataset        mode                                path  \
41  xception  cityscapes  RGB_HHAmid  ../models/temp/models/cityscape...   

      value  
41  0.67721  
     model     dataset mode                                path    value
49  resnet  cityscapes  RGB  ../models/temp/models/cityscape...  0.75386
     model     dataset  mode                                path     value
64  resnet  cityscapes  RGBD  ../models/temp/models/cityscape...  0.769716
     model     dataset        mode                                path  \
38  resnet  cityscapes  RGB_HHAmid  ../models/temp/models/cityscape...   

       value  
38  0.772335  
     model     dataset        mode    

In [74]:
indices

[57, 67, 49, 64, 89, 31]